In [1]:
class Block:
    
    def __init__(self,name):
        self.name = name
    
    def render(self, i):
        i += 1
        return "X{}[{}]".format(i, self.name), i
        
class ParallelBlock:
    def __init__(self, name, components):
        self.name = name
        self.components = components
    
    def render(self, i):
        start_i = i
        out = ""
        
        nodes = []
        
        # Render components
        for component in self.components:
            component, i = component.render(i)
            out += component
            out += "\n"
            nodes.append(i)
        end_i = i
        
        # Render invisible connections
        for ix1, ix2 in zip(nodes[:-1], nodes[1:]):
            out += "X{} ~~~ X{}".format(ix1, ix2)
            out += "\n"
        
        
        # Subgraph it
        i += 1
        out = "subgraph X{}[{}]\ndirection LR\n".format(i, self.name) + out
        
        out += "end"
        
        return out, i
        
class StackBlock:
    def __init__(self, name, components):
        self.name = name
        self.components = components
        
    def render(self, i):
        start_i = i
        out = ""
        
        nodes = []
        
        # Render components
        for component in self.components:
            component, i = component.render(i)
            out += component
            out += "\n"
            nodes.append(i)
        end_i = i
        
        # Render connections
        for ix1, ix2 in zip(nodes[:-1], nodes[1:]):
            out += "X{}-->X{}".format(ix1, ix2)
            out += "\n"
        
        # Subgraph it
        i += 1
        out = "subgraph X{}[{}]\ndirection TB\n".format(i, self.name) + out
        out += "end"
        
        return out, i
    
def write_simulation_block(sim_block):
    out = """```mermaid\ngraph TB\n"""
    out += sim_block.render(0)[0]
    out += "\n```"
    return out

In [2]:
# Block primitives
block_a = Block(name='A')
block_b = Block(name='B')
block_c = Block(name='C')
block_d = Block(name='D')
block_e = Block(name='E')

# Composite blocks for simulation 3
parallel_block = ParallelBlock('Parallel Block', [block_a, block_b])
stack_block = StackBlock('Stack Block', [block_c, block_d])

#Simulation Blocks
simulation1_block = StackBlock('Simulation 1', [block_a, block_b, block_c])
simulation2_block = ParallelBlock('Simulation 2', [block_a, block_b, block_c])
simulation3_block = StackBlock("Simulation 3", [parallel_block, stack_block, block_e])

In [3]:
print(write_simulation_block(simulation1_block))
print(write_simulation_block(simulation2_block))
print(write_simulation_block(simulation3_block))

```mermaid
graph TB
subgraph X4[Simulation 1]
direction TB
X1[A]
X2[B]
X3[C]
X1-->X2
X2-->X3
end
```
```mermaid
graph TB
subgraph X4[Simulation 2]
direction LR
X1[A]
X2[B]
X3[C]
X1 ~~~ X2
X2 ~~~ X3
end
```
```mermaid
graph TB
subgraph X8[Simulation 3]
direction TB
subgraph X3[Parallel Block]
direction LR
X1[A]
X2[B]
X1 ~~~ X2
end
subgraph X6[Stack Block]
direction TB
X4[C]
X5[D]
X4-->X5
end
X7[E]
X3-->X6
X6-->X7
end
```
